In [1]:
%matplotlib inline

import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import os
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

plt.style.use('fig_publication.mplstyle')
%config InlineBackend.figure_format = 'retina'

# Load fakes (sims) DF and DR4 DF

In [2]:
dr4_df = pd.read_csv("./dataframes/orig_dr4_df.csv.gzip", compression='gzip', index_col=0)

In [3]:
dr4_df

,amplitude_magn_g,anderson_darling_normal_magn_g,beyond_1_std_magn_g,beyond_2_std_magn_g,cusum_magn_g,inter_percentile_range_2_magn_g,inter_percentile_range_10_magn_g,inter_percentile_range_25_magn_g,kurtosis_magn_g,linear_fit_slope_magn_g,...,standard_deviation_magn_r,stetson_K_magn_r,weighted_mean_magn_r,anderson_darling_normal_flux_r,cusum_flux_r,excess_variance_flux_r,kurtosis_flux_r,mean_variance_flux_r,skew_flux_r,stetson_K_flux_r
sid,,,,,,,,,,,,,,,,,,,,,
245101100000933,0.177000,3.696596,0.370000,0.030000,0.158870,0.341499,0.286501,0.158501,-0.787381,-0.000212,...,0.097860,0.851286,15.842624,2.711263,0.099072,0.007498,-0.944294,0.087656,-0.507748,0.844530
245101100009703,0.293500,2.724011,0.290000,0.060000,0.184453,0.523001,0.333000,0.104000,0.201318,-0.001444,...,0.130134,0.869721,15.653200,4.130320,0.156431,0.013800,-1.050300,0.118238,-0.059173,0.909857
245101100010242,0.328500,5.493390,0.190000,0.050000,0.229448,0.559999,0.220499,0.094999,5.286414,-0.001808,...,0.068449,0.672620,16.533829,4.853297,0.187999,0.004273,8.948397,0.067449,2.465220,0.683464
245104200000799,0.329500,2.502895,0.467290,0.000000,0.089224,0.593760,0.490400,0.350000,-1.362500,0.000242,...,0.123850,0.858237,17.937359,1.524028,0.131790,0.011540,-0.980239,0.112776,-0.026306,0.870056
245104200000939,2.293000,12.605539,0.066038,0.056604,0.177457,4.114202,1.156300,0.426999,9.274009,-0.003301,...,0.394907,0.789900,15.854793,1.492581,0.322914,0.081920,0.337605,0.286570,0.109400,0.675794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597113100010642,0.483999,1.274301,0.293578,0.055046,0.187018,0.595680,0.369799,0.157999,1.613005,-0.001550,...,0.107312,0.680863,18.940697,5.417244,0.233112,0.008082,5.852146,0.105139,1.738649,0.670839
1597113100019088,0.323501,1.453954,0.271186,0.076271,0.202899,0.490780,0.252100,0.097000,1.162233,-0.001054,...,0.079516,0.702109,17.359133,3.377403,0.154508,0.005559,5.974216,0.076836,1.682657,0.710270
1597113100031221,0.074500,0.266636,0.307692,0.025641,0.153808,0.093640,0.060800,0.032499,1.197538,-0.000101,...,0.031295,0.753234,13.399863,0.861085,0.155385,0.000762,1.899416,0.029034,0.619902,0.756356


In [4]:
OID = 821107100005285

oid_df = dr4_df[dr4_df.index == OID]
oid_df

,amplitude_magn_g,anderson_darling_normal_magn_g,beyond_1_std_magn_g,beyond_2_std_magn_g,cusum_magn_g,inter_percentile_range_2_magn_g,inter_percentile_range_10_magn_g,inter_percentile_range_25_magn_g,kurtosis_magn_g,linear_fit_slope_magn_g,...,standard_deviation_magn_r,stetson_K_magn_r,weighted_mean_magn_r,anderson_darling_normal_flux_r,cusum_flux_r,excess_variance_flux_r,kurtosis_flux_r,mean_variance_flux_r,skew_flux_r,stetson_K_flux_r
sid,,,,,,,,,,,,,,,,,,,,,
821107100005285,1.0665,3.144564,0.367347,0.006803,0.41085,1.9118,1.548,0.855251,-0.911194,0.002203,...,0.485432,0.896099,19.664368,13.618182,0.398289,0.232481,0.133564,0.492454,1.121968,0.773862


In [5]:
#fakes_df = pd.concat([fakes_df, oid_df])
#fakes_df

# PCA 

In [6]:
def applypca(dataset_real, dataset_fakes, pca_num):
    X_real = dataset_real
    X_fakes = dataset_fakes
    
    ss = StandardScaler()
    scaled_X_real = ss.fit_transform(X_real)
    pca = PCA(pca_num)
    
    # apply same PCA to both real and fakes data
    pcs_X_real = pca.fit_transform(scaled_X_real)
    pcs_X_fakes = pca.transform(ss.transform(X_fakes))
    
    score = pcs_X_real[:,0:2]
    coeff = np.transpose(pca.components_[0:2, :])
    
    return pca, pcs_X_real, pcs_X_fakes, score, coeff

In [7]:
def pcaplot(score, coeff, datacolor):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.scatter(xs * scalex,ys * scaley, c=datacolor, s=6, alpha=0.6) 
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1], head_width= .003, color = 'r',alpha = 0.5)
        fakes_df_cols_latex_read = [r.replace('_', '\_') for r in fakes_df.columns.values]
        plt.text(coeff[i,0]* 1.12, coeff[i,1] * 1.12, list(fakes_df_cols_latex_read)[i], 
                     color = 'darkblue', ha = 'center', va = 'center', fontsize = 10)

In [8]:
pca, pcs_X_real, pcs_X_fakes, score, coeff = applypca(dataset_real=dr4_df, 
                                                      dataset_fakes=oid_df, 
                                                      pca_num=25)

# Build DFs from first (15) PCs

In [9]:
pcs_X_oid_df = pd.DataFrame(data=pcs_X_fakes, index=oid_df.index)
pcs_X_oid_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
sid,,,,,,,,,,,,,,,,,,,,,
821107100005285,25.132661,-4.287619,4.176179,9.490577,0.282622,-0.655225,-10.237013,-0.275212,2.065428,-0.98515,...,-0.041829,1.431878,0.636008,0.902594,-0.665632,3.140877,0.729018,3.032291,0.941942,1.362524


In [10]:
pcs_X_real_df = pd.DataFrame(data=pcs_X_real, index=dr4_df.index)
pcs_X_real_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
sid,,,,,,,,,,,,,,,,,,,,,
245101100000933,1.848374,-4.414712,-5.619388,5.260008,-0.999673,1.886673,2.324752,0.254310,-0.610019,-0.509973,...,2.364142,-2.795532,2.418044,-0.047208,1.244586,0.238595,0.737745,0.457679,1.083256,-0.968905
245101100009703,2.967064,-1.671175,-6.837338,3.595931,4.496301,0.035450,7.659707,0.350638,1.742033,-1.395584,...,1.982632,-2.632861,1.932105,-0.276466,0.654405,0.194872,0.558962,-0.063832,0.043463,0.311103
245101100010242,-0.108842,2.405274,-5.404214,0.148658,7.597006,0.413204,4.990583,0.844008,0.302768,-0.536326,...,0.503383,-1.557005,2.006571,0.340851,0.436470,-1.210648,-0.248009,-0.317351,0.133964,-0.095024
245104200000799,7.365672,-5.579897,-5.080349,6.771788,0.701663,4.046204,2.639507,-1.075667,-2.850329,-0.918465,...,3.195308,-5.855861,3.254418,1.645691,1.876212,3.695432,1.032424,0.009800,-0.009154,1.227088
245104200000939,30.154807,10.945092,-8.899698,12.322109,17.629009,-5.978311,18.177034,3.577059,2.805127,-3.233616,...,-4.799962,-11.330782,6.791227,9.284648,4.871954,8.505982,2.207025,-3.161091,-1.986230,-2.156008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597113100010642,3.946898,2.210204,-5.614792,0.057015,10.194058,-0.193864,7.076343,-0.936117,-2.615121,-0.239100,...,1.536582,-3.627772,1.341563,0.650310,1.440396,2.412695,0.874152,0.474602,0.765807,-0.876002
1597113100019088,0.413334,0.540809,-4.052421,-0.078610,4.553595,-0.578822,2.776984,0.468294,-0.761871,-0.078940,...,0.116539,-1.876106,0.258099,0.021929,-0.075823,-0.956434,0.471134,-0.058022,0.823703,-0.121341
1597113100031221,-5.107737,-2.974856,-3.773432,1.666605,-0.358642,0.263065,0.706929,2.276378,0.666725,-0.080482,...,0.428150,-1.025340,0.831355,0.225887,0.394189,0.765990,-0.194844,-0.448454,-0.265535,-0.598979


In [11]:
%%time

# KDTREE
from scipy.spatial import cKDTree

X = np.vstack(pcs_X_oid_df.values)
kdtree = cKDTree(np.vstack(pcs_X_real_df.values))
ktot=200
dist, ind = kdtree.query(X, k=ktot)

# Save PCA kdtree as pickle
#pickle.dump(kdtree,open('./pca_15pcs_orig_kdtree.p','wb'))

CPU times: user 2.75 s, sys: 234 ms, total: 2.98 s
Wall time: 3 s


In [12]:
k_neighbor = 1   # change this number to find the NN you want

print(dist[:, k_neighbor])   # drop id; assumes sorted -> see args!
print(ind[:, k_neighbor])    # drop id

[7.9707464]
[98525]


In [13]:
kdtree_results_df = pd.DataFrame()
kdtree_results_df['sid'] = pcs_X_oid_df.index
kdtree_results_df = kdtree_results_df.set_index('sid')

kdtree_dist_df = kdtree_results_df.copy()

for knn in range(0,ktot):    
    dr4_match_oid, dr4_match_dist = [],[]
    for ix, dis in zip(ind[:, knn], dist[:, knn]):
        dr4_match_oid.append(dr4_df.iloc[ix].name)
        dr4_match_dist.append(dis)
    kdtree_results_df[f'knn={knn+1}_match_OID'] = dr4_match_oid
    kdtree_dist_df[f'knn={knn+1}_dist'] = dr4_match_dist

In [14]:
#kdtree_results_df.to_csv("./dataframes/pca_15pcs_orig_kdtree_results_df.csv.gzip", compression='gzip')
kdtree_results_df

,knn=1_match_OID,knn=2_match_OID,knn=3_match_OID,knn=4_match_OID,knn=5_match_OID,knn=6_match_OID,knn=7_match_OID,knn=8_match_OID,knn=9_match_OID,knn=10_match_OID,...,knn=191_match_OID,knn=192_match_OID,knn=193_match_OID,knn=194_match_OID,knn=195_match_OID,knn=196_match_OID,knn=197_match_OID,knn=198_match_OID,knn=199_match_OID,knn=200_match_OID
sid,,,,,,,,,,,,,,,,,,,,,
821107100005285,821107100005285,582109100008627,724101100027872,633108300023697,587106400020085,635104300022736,793105100016413,667113400000692,725116100002465,813105100004493,...,799111400025654,683106100033432,646113200001492,677101100002183,752113100002574,681106200027577,681106400000714,682113300013408,821115300012972,817103100001382


In [15]:
#kdtree_dist_df.to_csv("./dataframes/pca_15pcs_orig_kdtree_dist_df.csv.gzip", compression='gzip')
kdtree_dist_df

,knn=1_dist,knn=2_dist,knn=3_dist,knn=4_dist,knn=5_dist,knn=6_dist,knn=7_dist,knn=8_dist,knn=9_dist,knn=10_dist,...,knn=191_dist,knn=192_dist,knn=193_dist,knn=194_dist,knn=195_dist,knn=196_dist,knn=197_dist,knn=198_dist,knn=199_dist,knn=200_dist
sid,,,,,,,,,,,,,,,,,,,,,
821107100005285,0.021654,7.970746,8.175978,8.29515,8.330584,8.632303,9.149251,9.272597,9.502597,9.611685,...,13.502166,13.52317,13.524829,13.530399,13.531816,13.534004,13.536636,13.559938,13.560165,13.563705


In [16]:
# Total real objects matched
print("Total real objects matched:", len(np.hstack(kdtree_results_df.values)))

# Of above, total UNIQUE real objects matched
print("Total real UNIQUE objects matched:", len(np.unique(np.hstack(kdtree_results_df.values))))
print(np.unique(np.hstack(kdtree_results_df.values)))

Total real objects matched: 200
Total real UNIQUE objects matched: 200
[443116100000521 536107100007124 545114100008154 578101400000953
 582109100008627 587101300021768 587103400035771 587105200023971
 587106400017817 587106400020085 587110200030917 587111200031270
 587112100025293 587114300001085 594109100004321 625101400029189
 626107400004433 632209400014771 633104100005749 633105400014971
 633106200025605 633108300023697 633109100025896 633111400007672
 633113400015478 634108100005436 634110300013002 635104300022736
 635105200019309 635105300019099 635106300013599 635111400000448
 636103300014529 636104100033344 636113400034409 643113300004423
 646113200001492 667113400000692 668106100003131 672115100001897
 672115200000744 673103100001638 673103400002781 673107200003416
 676102100009347 676102300004634 676102400000856 676104100007808
 676108200017280 677101100002183 677103200002903 679114300023795
 681101100015804 681101400016361 681106200027577 681106400000714
 682101200013786 68

# Visually inspect matched OIDs

In [17]:
from IPython.display import display_markdown

print([display_markdown(f'https://ztf.snad.space/view/{oid}', raw=True) for oid in \
         np.unique(np.hstack(kdtree_results_df.values))])

https://ztf.snad.space/view/443116100000521

https://ztf.snad.space/view/536107100007124

https://ztf.snad.space/view/545114100008154

https://ztf.snad.space/view/578101400000953

https://ztf.snad.space/view/582109100008627

...

https://ztf.snad.space/view/849102200015533

https://ztf.snad.space/view/861109300000431

https://ztf.snad.space/view/863111400001088

https://ztf.snad.space/view/865116100005870

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [18]:
from IPython.display import display_markdown

for knn in range(0,ktot):
    print(f"KNN {knn+1} \n")
    _ = [display_markdown(f'https://ztf.snad.space/view/{oid}', raw=True) for oid in \
         kdtree_results_df[f'knn={knn+1}_match_OID']]

KNN 1 



https://ztf.snad.space/view/821107100005285

KNN 2 



https://ztf.snad.space/view/582109100008627

KNN 3 



...

https://ztf.snad.space/view/682113300013408

KNN 199 



https://ztf.snad.space/view/821115300012972

KNN 200 



https://ztf.snad.space/view/817103100001382

https://ztf.snad.space/view/587106400017817 AGN
https://ztf.snad.space/view/633105400014971 AGN
https://ztf.snad.space/view/635105200019309 SN (template image taken during SN, see https://alerce.online/object/ZTF18aakzlnq)
https://ztf.snad.space/view/635111400000448 (template image taken during SN, see https://alerce.online/object/ZTF18aamnquz)
https://ztf.snad.space/view/676102400000856 SN, not in Alert stream
https://ztf.snad.space/view/676104100007808 SN/AGN, not in Alert stream
https://ztf.snad.space/view/681106400000714 SN/AGN, not in Alert stream
https://ztf.snad.space/view/783109400002438 SN
https://ztf.snad.space/view/788105100004444 SN?/AGN
https://ztf.snad.space/view/794105300002705 SN, not in Alert stream
https://ztf.snad.space/view/798101400000531 Double peak SN!(maybe!) (see https://antares.noirlab.edu/loci/ANT2018dbixe)
https://ztf.snad.space/view/822101200002926 SN
https://ztf.snad.space/view/823101400006642 SN, not in Alert stream